# ReACT Agents in Langgraph

- Standard Langgraph agents can be nodes and edges, where you can write basic nodes, conditional nodes, conditional edges and what not. 
- Writing agents that makes decisions is where you give away certain control of the agents flow to the llm, that decides what tools to call. 
- Certain models have capabilities to make tool calls. 
- The llms can make tool calls based on arguments defined in the arguments and description in the docstring. 

In [ ]:
from typing import Annotated
from typing import Any, Optional , List
from typing_extensions import TypedDict
import os, json, logging
from neo4j import GraphDatabase
import uuid
#Langgraph and Langchain dependencies. 
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
#from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.runnables.graph_mermaid import draw_mermaid_png
import ast

#Manage Environment Variables. 
from dotenv import load_dotenv

load_dotenv()
auradb_conn=os.environ["AURADB_CONN"]
auradb_username = os.environ["AURADB_USERNAME"]
auradb_password = os.environ["AURADB_PASSWORD"]

In [233]:
import os , getpass
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

def _set_env(var:  str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")





In [234]:
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    messages: Annotated[list, add_messages]


In [282]:
@tool("create_nodes")
def create_nodes(graph:str) -> str:

    """
    From the Logical graph generated by process passage from the input passage, this tool creates two list objects, nodes and relationships, which will
    be passed to `push_graph` which will push them to the graph database auradb
    
    args:
        graph (str): A serialized json object which is generated by process_passage, which is a logical graph inferred from the input passage


    returns:
        str: Returns a serialized json with a list of nodes and relationships between those nodes. 
    """
    
    loaded = json.loads(graph)
    print(loaded)
    print(type(loaded))
    input_data = loaded["graph"] if "graph" in loaded else loaded
    
    print(type(input_data))
    print(input_data)

    # Transform nodes
    nodes = {item["node"]: {"id": str(uuid.uuid4())} for item in input_data}

    # Transform relationships
    relationships = []
    for item in input_data:
        source_node = item["node"]
        for rel in item["relationships"]:
            relationships.append({
                "source": source_node,
                "target": rel.get("target", rel.get("source")),
                "type": rel["type"].upper().replace(" ", "_")
            })
    return json.dumps({"nodes":nodes,"relationships":relationships})

In [283]:
@tool("push_graph")
def push_graph(graph: str) -> str:

    """  
    Pushes the Serialized JSON string object with nodes and relationships to an AuraDB Graph database instance. Serialized json is produced by
    the `create_nodes` tool.
    
    Args:
        graph (str): A Serialized JSON object as a string containing a list of nodes and relationships created by the create_nodes tool.
    Returns:
        str: A serialized string of the summary of objects successfully pushed to AuraDB
    """

    AURA_CONNECTION_URI = auradb_conn
    AURA_USERNAME = "neo4j"
    AURA_PASSWORD = auradb_password

    input_data = json.loads(graph)
    nodes = input_data["nodes"]
    relationships = input_data["relationships"]
    # Driver instantiation
    driver = GraphDatabase.driver(
        AURA_CONNECTION_URI,
        auth=(AURA_USERNAME, AURA_PASSWORD)
    )


    def create_graph(tx):
        # Create nodes with unique IDs
        for name, props in nodes.items():
            nodes_result = tx.run("""
                MERGE (n:Species {name: $name})
                SET n.id = $id
            """, name=name, id=props["id"])

        # Create relationships
        for rel in relationships:
            relationships_result = tx.run("""
                MATCH (a:Species {name: $source}), (b:Species {name: $target})
                MERGE (a)-[r:RELATES_TO {type: $type}]->(b)
            """, source=rel["source"], target=rel["target"], type=rel["type"])
    
        return {"nodes": nodes_result.consume(), "relationships": relationships_result.consume()}

    
    try:
        with driver.session() as session:
            summary = session.execute_write(create_graph)
        
            return json.dumps({"response":"Graph Push to AuraDB successfull", "summary": str(summary)})
        
    except Exception as e:
        return json.dumps({"response":"Graph Push to AuraDB Fialed", "summary": str(summary)})

In [284]:
@tool("process_passage")
def process_passage(text:str) -> str:
    
    """
    Identifies node and relationships between entities in the input text passage.
    Args:
        text (str): The passage to parse and retrieve nodes and relationships from.
    Returns:
        str: A Serialized JSON string consisting a list of nodes and relationships this 
            is to be structured by push_graph tool and uploaded to the graph database.
    """


    prompt = PromptTemplate.from_template("""For the following passage, output ONLY a complete, valid serialized JSON array of node objects as described below.
        Do not add any explanation or comments. Output must parse with json.loads().  
    For example a graph relationship: Lion --> (predator) --> Antelope should output a serialized json string \n

    the passage starts from the next line \n {text} \n


    *** Requirements ***

        - Only return the json object. Nothing else.
        - each node relationship should have only one source or target. Not a list. 
        - Add any node and relationship properties (ecosystem, etc.) based on the passage

    *** Example output for a relationship ***

    {{
       "node": "Jaguar",
       "relationships": [
         {{
           "type": "predator",
           "target": "Capybara"
         }}
       ]
     }}
                                          
    Stick to the schema given.

    """)
    
    parser_llm = ChatOllama(model="mistral:instruct", max_tokens=8192)
    json_parser = JsonOutputParser()
# Wrap it with OutputFixingParser
    fixing_parser = OutputFixingParser.from_llm(parser=json_parser, llm=parser_llm)

    output_schema = { "type": "array",
                      "items": { 
                  "type": "object",
                  "properties": 
                      { "node": { "type": "string" },
                            "relationships": { "type": "array",
                                  "items": { "type": "object",
                                          "properties": { "type": { "type": "string" },
                                                  "target": { "type": "string" } },
                                                  "required": ["type", "target"] } 
                                                },
                                                "ecosystem": { "type": "string" } 
                                                },
                                                "required": ["node", "relationships", "ecosystem"] } }
    
    structured_llm = parser_llm.with_structured_output(output_schema)

    formatted_prompt = prompt.format(text=text)
    response = structured_llm.invoke(formatted_prompt)
    print(response)
    
    try:
       
       loaded = json.dumps(response)
       print(loaded)
       return json.dumps({"graph":response})
    except Exception as e:
        
        return json.dumps({"messages":"Graph creation failed. Try again."})
    

In [294]:
llm = ChatOpenAI(model="o3-mini")
tools = [process_passage, create_nodes, push_graph]
llm_with_tools = llm.bind_tools(tools)

In [295]:
def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]

    if (len(last_message.tool_calls) > 1):
        return "tool" 
    else:
        return "end"

In [296]:
from langchain_core.runnables import RunnableConfig
import pprint

def agent(
        state:State, 
        config:RunnableConfig
        ):
    system_prompt = SystemMessage(
        """You are a helpful assistant responsible for helping users with their requests by using whichever tools are available."""
    )
    response = llm_with_tools.invoke([system_prompt] + state["messages"], config)
    pprint.pprint(response)
    return {"messages": [response]}

In [297]:
tools_by_name = {tool.name: tool for tool in tools}
def call_tool(state:State):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content = json.dumps(tool_result),
                name = tool_call["name"],
                tool_call_id =  tool_call["id"]
            )
        )
    return {"messages": outputs}

In [298]:
from langgraph.prebuilt import tools_condition, ToolNode
def router(state:State):

    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tools"
    
    else:
        return "end"

In [299]:
from langgraph.graph import StateGraph , END

workflow = StateGraph(State)
workflow.add_node("agent", agent)
workflow.add_node("tools", call_tool)   
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    router,
    {
        "tools": "tools",
        "end": END
    }
)

workflow.add_edge("tools","agent")

agent = workflow.compile()

In [271]:
from IPython.display import Image, display
import mermaid as md 
from mermaid.graph import Graph

mermaid_code = agent.get_graph().draw_mermaid()
render = md.Mermaid(mermaid_code)

render


In [300]:
passage = """
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also preys on the Sloth (Bradypus variegatus), which has a mutualistic relationship with several species of algae, such as Trentepohlia spp., that grow on its fur, providing camouflage. Additionally, the Sloth shares a symbiotic relationship with the Moth (Cryptoses choloepi), which uses the sloth’s fur for habitat.
From an evolutionary standpoint, the Capybara and the Jaguar belong to different orders: Rodentia and Carnivora, respectively, reflecting deep phylogenetic divergence. The Sloth is taxonomically grouped within the order Pilosa, distinct from the other mammals mentioned.
Together, these species form a complex web of predation, competition, symbiosis, and shared evolutionary history, making the tropical rainforest a dynamic and interconnected ecosystem.
"""

In [301]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [HumanMessage(content=f"Process the following passage ONCE, identify the relationships between species and push it to the auradb graph database then exit, {passage}")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Process the following passage ONCE, identify the relationships between species and push it to the auradb graph database then exit, 
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also p